In [1]:
import pandas as pd

In [45]:

import folium
from IPython.display import IFrame
import pandas as pd
import numpy as np


data1 = pd.read_csv("data1.csv")
data2 = pd.read_csv("data2.csv")

# Merge datasets
merged_df = pd.merge(data1, data2, on='listed_incity', how='left')


italian_df = merged_df[merged_df['cuisines'].str.contains("Italian", case=False, na=False)]


italian_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)


for _, row in italian_df.iterrows():
    if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['listed_incity']} | {row['cuisines']} | Rating: {row['rate']}⭐",
            icon=folium.Icon(color='purple')
        ).add_to(italian_map)


italian_map.save("italian_restaurants_map.html")


IFrame(src='italian_restaurants_map.html', width=800, height=600)

In [ ]:

import folium
from folium.plugins import MarkerCluster
from IPython.display import IFrame
import pandas as pd
import numpy as np


data1 = pd.read_csv("data1.csv")
data2 = pd.read_csv("data2.csv")


merged_df = pd.merge(data1, data2, on='listed_incity', how='left')



restaurant_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)


marker_cluster = MarkerCluster().add_to(restaurant_map)


for _, row in merged_df.iterrows():
    if not np.isnan(row['Latitude']) and not np.isnan(row['Longitude']):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['listed_incity']} | Rating: {row['rate']} | Cost for Two: ₹{row['approx_costfor_two_people']}",
            icon=folium.Icon(color='blue')
        ).add_to(marker_cluster)


restaurant_map.save("restaurant_density_map.html")


IFrame(src='restaurant_density_map.html', width=800, height=600)

In [120]:


import pandas as pd



df = pd.read_csv('data1.csv')


df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


print("Columns in dataset:", df.columns.tolist())


df['rate'] = df['rate'].astype(str).str.extract(r'(\d+\.\d+)')
df['rate'] = pd.to_numeric(df['rate'], errors='coerce')


cost_col = next((col for col in df.columns if 'cost' in col), None)

if cost_col:
    df[cost_col] = df[cost_col].astype(str).str.replace(',', '', regex=False)
    df[cost_col] = pd.to_numeric(df[cost_col], errors='coerce')

# 1. Shape of the dataset
print("1. Shape of the dataset:", df.shape)

# 2. Restaurants serving North Indian
north_indian_count = df['cuisines'].astype(str).str.contains("North Indian", case=False, na=False).sum()
print("2. Restaurants serving North Indian:", north_indian_count)

# 3. Most common cuisine
top_cuisine = df['cuisines'].astype(str).str.split(', ').explode().value_counts().idxmax()
print("3. Most commonly offered cuisine:", top_cuisine)

df['approx_costfor_two_people'] = pd.to_numeric(df['approx_costfor_two_people'], errors='coerce')


filtered_data = df.dropna(subset=['approx_costfor_two_people', 'listed_incity'])


localities = ['Banashankari', 'Church Street', 'Indiranagar', 'Whitefield']

filtered_data = filtered_data[filtered_data['listed_incity'].isin(localities)]

avg_cost_by_city = filtered_data.groupby('listed_incity')['approx_costfor_two_people'].mean()

highest_avg_city = avg_cost_by_city.idxmax()
highest_avg_cost = avg_cost_by_city.max()

print(f"4. Locality with the highest average cost for dining (for two people): {highest_avg_city}")
print(f"Average cost: ₹{highest_avg_cost:.2f}")
# 5. Top rated restaurant type with >1000 votes
q5_df = df[df['votes'] > 1000]
if 'rest_type' in df.columns:
    top_rest_type = q5_df.groupby('rest_type')['rate'].mean().sort_values(ascending=False)
    print("5. Top rated restaurant type (>1000 votes): rest_type")
    print(top_rest_type.head(1))

# 6. Minimum cost to eat out
merged_df['approx_costfor_two_people'] = pd.to_numeric(
    merged_df['approx_costfor_two_people'].astype(str).str.replace(',', ''), errors='coerce'
)
print("6.Minimum cost for two people:", merged_df['approx_costfor_two_people'].min())

#7
# Filter the dataset for Banashankari restaurants
banashankari_data = df[df['listed_incity'] == 'Banashankari']

banashankari_online_orders = banashankari_data[banashankari_data['online_order'] == 'Yes']

total_online_orders = df[df['online_order'] == 'Yes'].shape[0]

percentage_banashankari = (banashankari_online_orders.shape[0] / total_online_orders) * 100

print(f"7. Percentage of total online orders received by restaurants in Banashankari: {percentage_banashankari:.2f}%")

filtered_data = df[(df['votes'] > 500) & (df['rate'] < 3.0)]

restaurant_counts_by_location = filtered_data['listed_incity'].value_counts()

top_locality = restaurant_counts_by_location.idxmax()
top_locality_count = restaurant_counts_by_location.max()

print(f"8. Locality with the most restaurants (over 500 votes and rating below 3.0): {top_locality} ({top_locality_count} restaurants)")

restaurant_type_diversity = df.groupby('listed_incity')['rest_type'].nunique()

sorted_localities = restaurant_type_diversity.sort_values(ascending=False)

top_locality = sorted_localities.idxmax()
top_diversity_count = sorted_localities.max()

print(f"9.Locality with the most diverse restaurant types: {top_locality} ({top_diversity_count} types)")

merged_df['approx_costfor_two_people'] = (
    merged_df['approx_costfor_two_people']
    .astype(str)
    .str.replace(',', '', regex=False)
    .astype(float)
)


buffet = merged_df[merged_df['rest_type'].str.contains('Buffet', na=False)]
avg_cost_buffet = buffet['approx_costfor_two_people'].mean()


delivery = merged_df[merged_df['rest_type'].str.contains('Delivery', na=False)]
avg_cost_delivery = delivery['approx_costfor_two_people'].mean()

cost_difference = abs(avg_cost_buffet - avg_cost_delivery)

print(f"10. Buffet Avg Cost: ₹{avg_cost_buffet:.2f}")
print(f"Delivery Avg Cost: ₹{avg_cost_delivery:.2f}")
print(f"Cost Difference: ₹{cost_difference:.2f}")


# 11. Max votes with online ordering
if 'online_order' in df.columns:
    online_votes = df[df['online_order'] == 'Yes']['votes'].max()
    print("11. Max votes (online order):", online_votes)


both_cuisines_data = df[(df['cuisines'].str.contains('North Indian', case=False, na=False)) & 
                          (df['cuisines'].str.contains('Chinese', case=False, na=False))]


average_rating = both_cuisines_data['rate'].mean()

print(f"12.Average rating of restaurants that serve both North Indian and Chinese cuisines: {average_rating}")


df['estimated_revenue'] = df['votes'] * df['approx_costfor_two_people']


revenue_by_area = df.groupby('listed_incity')['estimated_revenue'].sum()


most_profitable_area = revenue_by_area.idxmax()
print(f"13.The most profitable area for Zomato based on potential revenue estimation is: {most_profitable_area}")

df['rate'] = pd.to_numeric(df['rate'], errors='coerce')

rest_type_ratings = df.groupby('rest_type')['rate'].mean().sort_values()

lowest_rated_rest_type = rest_type_ratings.idxmin()
lowest_rating = rest_type_ratings.min()

print("14.Restaurant type with the lowest average rating (needs focus):", lowest_rated_rest_type)
print("Average rating:", lowest_rating)

filtered_df = df[
    (df['rate'].astype(float) > 4.2) &
    (df['votes'].astype(int) > 500) &
    (df['online_order'] == 'Yes')

]

top_area = filtered_df['listed_incity'].value_counts().idxmax()

print("15. Best area for Zomato to invest:", top_area)


Columns in dataset: ['online_order', 'book_table', 'rate', 'votes', 'rest_type', 'dish_liked', 'cuisines', 'approx_costfor_two_people', 'listed_intype', 'listed_incity']
1. Shape of the dataset: (51717, 10)
2. Restaurants serving North Indian: 21085
3. Most commonly offered cuisine: North Indian
4. Locality with the highest average cost for dining (for two people): Church Street
Average cost: ₹771.99
5. Top rated restaurant type (>1000 votes): rest_type
rest_type
Bakery    4.8
Name: rate, dtype: float64
6.Minimum cost for two people: 40.0
7. Percentage of total online orders received by restaurants in Banashankari: 1.79%
8. Locality with the most restaurants (over 500 votes and rating below 3.0): Brookefield (8 restaurants)
9.Locality with the most diverse restaurant types: BTM (61 types)
10. Buffet Avg Cost: ₹nan
Delivery Avg Cost: ₹415.05
Cost Difference: ₹nan
11. Max votes (online order): 16832
12.Average rating of restaurants that serve both North Indian and Chinese cuisines: 3.572